In [4]:
 ratings = sc.textFile("/home/roshni/Desktop/jupyter/ratings.csv").\
  map(lambda x: x.split("::")).map(lambda y: y(int(y[0]),int(y[1]),int(y[1])))

In [5]:
 movies = sc.textFile("/home/roshni/Desktop/jupyter/movies.csv").\
  map(lambda x: x.split("::")).map(lambda y: y(int(y[0]),String(y[1]),String(y[1])))

In [8]:
ratings_Df=spark.read.csv("/home/roshni/Desktop/jupyter/ratings.csv", inferSchema = True, header = True)

In [9]:
movies_Df=spark.read.csv("/home/roshni/Desktop/jupyter/movies.csv", inferSchema = True, header = True)

In [10]:
splits = ratings.randomSplit([0.8, 0.2])

In [11]:
training = splits[0].cache()
test = splits[1].cache()

In [12]:
import array
import sys
from collections import namedtuple

from pyspark import SparkContext, since
from pyspark.rdd import RDD
from pyspark.mllib.common import JavaModelWrapper, callMLlibFunc, inherit_doc
from pyspark.mllib.util import JavaLoader, JavaSaveable
from pyspark.sql import DataFrame

In [13]:
import sys
from pyspark.ml.recommendation import ALS

In [17]:
training_RDD, validation_RDD, test_RDD = ratings.randomSplit([6, 2, 2], seed=0)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [19]:
seed = 5
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02


In [25]:
from pyspark.mllib.recommendation import ALS
import math

In [28]:
min_error = float('inf')
best_rank = -1
best_iteration = -1


In [30]:
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank

print 'The best model was trained with rank %s' % best_rank

In [ ]:
# the code is under work

# model = ALS.train(training_RDD, best_rank, seed=seed, iterations=iterations,
#                       lambda_=regularization_parameter)
# predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
# rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
# error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
# print 'For testing data the RMSE is %s' % (error)